In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [80]:
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51271_2024-05-12~2025-05-12.xlsx')

In [82]:
na_df = pd.read_csv('/Users/seanyu_mac/Downloads/네이버스마트스토어 공구 - 시트1 (5).csv')

In [84]:
today = datetime.today().strftime('%Y-%m-%d')

In [86]:
fs_df['주문 상태'].unique()

array(['구매확정', '취소완료'], dtype=object)

In [88]:
yaksa = fs_df['상품명'].unique()[0].split()[1].strip()
yaksa

'두둥실팜'

In [90]:
for i in fs_df.loc[~(fs_df['주문 상태'] == '구매확정')].index:
    data = fs_df.loc[i, '정산가'] * -1
    print(data)
    fs_df.loc[i, '정산가'] = data

fs_df.loc[~(fs_df['주문 상태'] == '구매확정'), '정산가']

-63000
-63000


1    -63000
14   -63000
Name: 정산가, dtype: int64

In [47]:
# 타 스마트 스토어 판매건이 없으면 바로 정산자료 추출
fs_df.to_excel(f'/Users/seanyu_mac/Downloads/{yaksa}_정산자료_{today}.xlsx')

In [92]:
fs_df.rename(columns={'정산가': '정산예정금액'}, inplace=True)
fs_df['네이버페이 주문관리 수수료'] = '해당 없음'

In [94]:
fs_df = fs_df[['결제 번호', '상품 번호', '주문 번호', '결제 완료일', '주문 상태', '상품명', '옵션명', '수량', '정상가',
       '판매가', '쿠폰 할인', '포인트 할인', '적립금', '네이버페이 주문관리 수수료', '정산예정금액', '수령인명', '수령인 연락처', '구매자명',
       '구매자 ID', '구매자 연락처', '주소', '우편번호', '배송 메모', '주문 확인일', '발주 요청일',
       '배송 완료일', '구매 확정일', '배송택배사', '송장 번호', '배송비 정책', '배송비', '도서산간 배송비',
       '취소 완료일', '판매자 취소일', '반품 요청일', '수거 요청일', '수거 완료일', '반품 완료일']]

In [96]:
na_df['정산예정금액'] = na_df['정산예정금액'].apply(lambda x: x[1:])
na_df['정산예정금액'] = na_df['정산예정금액'].apply(lambda x: x.replace(",", ""))
na_df['정산예정금액'] = na_df['정산예정금액'].apply(lambda x: int(x))

In [98]:
na_df['네이버페이 주문관리 수수료'] = na_df['네이버페이 주문관리 수수료'].astype(int)
na_df['네이버페이 주문관리 수수료']

0   -5408
1   -3234
2   -4867
Name: 네이버페이 주문관리 수수료, dtype: int64

In [100]:
na_df.columns = ['결제 번호', '주문번호', '결제 완료일', '구매 확정일', '상품 번호', '상품명', '옵션명', '수량', '상품가격', '옵션가격', '판매자 부담 할인액', '최종 상품별 총 주문금액', '정산예정금액', '네이버페이 주문관리 수수료', '매출연동 수수료', '마지막 유입경로']

In [102]:
na_df['상품가격'] = na_df['상품가격'].apply(lambda x: x[1:])
na_df['상품가격'] = na_df['상품가격'].apply(lambda x: x.replace(",", ""))
na_df['상품가격'] = na_df['상품가격'].apply(lambda x: int(x))
na_df

,결제 번호,주문번호,결제 완료일,구매 확정일,상품 번호,상품명,옵션명,수량,상품가격,옵션가격,판매자 부담 할인액,최종 상품별 총 주문금액,정산예정금액,네이버페이 주문관리 수수료,매출연동 수수료,마지막 유입경로
0,2025042799201221,2025042767453391,2025/04/27 18:20,2025/05/07 3:49,11725124354,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,선택: 8박스,1,256000,\0,"\107,000","\149,000",143592,-5408,0,네이버쇼핑 외
1,2025042826819031,2025042878753281,2025/04/28 8:24,2025/05/07 3:11,11725124354,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,선택: 5박스,1,256000,"-\50,000","\116,900","\89,100",95766,-3234,0,네이버쇼핑 외
2,2025042669341291,2025042647017671,2025/04/26 15:34,2025/05/07 3:10,11725124354,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,선택: 8박스,1,256000,\0,"\121,900","\134,100",144133,-4867,0,네이버쇼핑 외


In [104]:
na_df['판매가'] = na_df['정산예정금액'] - na_df['네이버페이 주문관리 수수료']

In [106]:
na_df['주문 상태'] = '구매확정'
na_df['판매처'] = '네이버스토어'
fs_df['판매처'] = '친한스토어'

In [108]:
common_cols = fs_df.columns.intersection(na_df.columns)
combined_df = pd.concat([fs_df[common_cols], na_df[common_cols]], ignore_index=True)
combined_df

,결제 번호,상품 번호,결제 완료일,주문 상태,상품명,옵션명,수량,판매가,네이버페이 주문관리 수수료,정산예정금액,구매 확정일,판매처
0,parma20250429230434KfUN1,51271,2025-04-29 23:05:05.302000,구매확정,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,3박스,1,63000,해당 없음,63000,2025-05-11 19:00:00.005000,친한스토어
1,parma20250429212411E9mZ1,51271,2025-04-29 21:25:32.107000,취소완료,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,3박스,1,63000,해당 없음,-63000,NaT,친한스토어
2,parma20250429102047K3gX3,51271,2025-04-29 10:21:41.725000,구매확정,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,5박스,1,99000,해당 없음,99000,2025-05-08 19:00:00.028000,친한스토어
3,parma20250428121657DFMv1,51271,2025-04-28 12:17:29.958000,구매확정,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,3박스,1,63000,해당 없음,63000,2025-05-07 19:00:00.028000,친한스토어
4,parma20250428010448ERlz9,51271,2025-04-28 01:05:02.086000,구매확정,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,5박스,1,99000,해당 없음,99000,2025-05-07 19:00:00.028000,친한스토어
5,parma20250427231621GfY66,51271,2025-04-27 23:16:45.721000,구매확정,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,3박스,1,63000,해당 없음,63000,2025-05-07 22:00:00.007000,친한스토어
6,parma20250427220235BQyg2,51271,2025-04-27 22:03:01.388000,구매확정,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,3박스,1,63000,해당 없음,63000,2025-05-07 16:00:00.008000,친한스토어
7,parma20250427203724Jpgt1,51271,2025-04-27 20:37:39.281000,구매확정,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,3박스,1,63000,해당 없음,63000,2025-05-07 22:00:00.007000,친한스토어
8,parma20250427142134JPon5,51271,2025-04-27 14:22:12.946000,구매확정,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,8박스,1,149000,해당 없음,149000,2025-05-07 19:00:00.028000,친한스토어
9,parma20250426162939BD9z9,51271,2025-04-26 16:29:49.437000,구매확정,[공동구매] 두둥실팜 x 유트리션 와일드 알래스카 오메가3 미니 60캡슐 / 3 5...,8박스,1,149000,해당 없음,149000,2025-05-07 16:00:00.008000,친한스토어


In [110]:
combined_df.to_excel(f'/Users/seanyu_mac/Downloads/{yaksa}_정산자료_{today}.xlsx')